In [ ]:
import boto3
import os 

# -----------------------------------
# Enter these values here:
thebucket = 'compass-us-origtv'
thefile = 'matching_svod_all'
# ------------------------------------
# Don't change anything under here.

ec2 = boto3.resource('ec2')
ec2client = boto3.client('ec2')
s3 = boto3.resource('s3')
s3client = boto3.client('s3')

# paginate 1000 at a time
paginator = s3client.get_paginator('list_object_versions')
pageresponse = paginator.paginate(Bucket=thebucket, Prefix=thefile)

# iter over the pages from the paginator
for pageobject in pageresponse:
    # Find if there are any delmarkers
    if 'DeleteMarkers' in pageobject.keys():
        for each_delmarker in pageobject['DeleteMarkers']:
            # Create a resource for the version-object 
            # and use .delete() to remove it.
            fileobjver = s3.ObjectVersion(
                thebucket,
                each_delmarker['Key'],
                each_delmarker['VersionId']
            )
            # I added this output just so I could watch the script run.
            print('Restoring ' + each_delmarker['Key'])
            # Lastly, lets remove the del marker and recover one of many files.
            fileobjver.delete()